In [101]:
!git add .
!git commit -m "implemented source credibility, ran labeling, and outlined feature/glove calculation and model training setup"
!git pull
!git push -u origin main

[main 179edca] implemented source credibility, ran labeling, and outlined feature/glove calculation and model training setup
 8 files changed, 11172 insertions(+), 273 deletions(-)
 create mode 100644 Sports_Misinformation_Tool/.ipynb_checkpoints/labeled_RSS_posts-checkpoint.csv
 create mode 100644 Sports_Misinformation_Tool/labeled_RSS_posts.csv
Already up to date.
branch 'main' set up to track 'origin/main'.


To github.com:thernandez7/Sports_Misinformation_Tool.git
   6f1abeb..179edca  main -> main


## Sports Misinformation Classification Tool

In [1]:
import mysql.connector
from sqlalchemy import create_engine
import pandas as pd

### Connect to the Database

In [18]:
username = 'root'  
password = 'password'
host = 'localhost' 
database = 'sports_news_db'

connection = mysql.connector.connect(
    host=host,
    user=username,
    password=password,
    database=database
)

if connection.is_connected():
    print("Connected to the database successfully!")

engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{host}/{database}", echo=False)

Connected to the database successfully!


#### Use this format to insert into the database

INSERT INTO articles (team_or_player, source, publication_date, content, trust_score, classification, link)
VALUES
('New York Yankees, Los Angeles Lakers', 'ESPN', '2024-10-27', 'Yankees article content example.', 85.00, 'real', 'https://example.com/article1'),
('Los Angeles Lakers', 'Twitter', '2024-10-27', 'Lakers article content example.', 60.00, 'fake', 'https://example.com/article2');


#### Table fields 
Table Name: articles

Fields: 

     id INT AUTO_INCREMENT PRIMARY KEY,
     
     team_or_player VARCHAR(500), (This will be the article title that we can query- usually includes teams or names in it)
     
     source VARCHAR(200),
     
     publication_date DATE,
     
     content TEXT,
     
     trust_score DECIMAL(5, 2), 
     
     classification ENUM('credible', 'uncredible', 'unknown') DEFAULT 'unknown',

     link VARCHAR(255)

### Simulate Tool Working

In [118]:
team_or_player = input("Enter the team or player's name: ")

query = f"SELECT * FROM articles WHERE team_or_player LIKE '%{team_or_player}%'" #search for entered name/team in the title 
df_result = pd.read_sql(query, con=engine, params={'team_or_player': team_or_player})

#get results
if not df_result.empty:
    print(f"Articles related to {team_or_player}:")
    display(df_result)
else:
    print(f"No articles found for {team_or_player}.")

Enter the team or player's name:  x


No articles found for x.


### Get Article Entries from RSS Feeds

In [5]:
# # reddit API - collected 1876 posts
# import csv
# import praw
# from datetime import datetime


# reddit = praw.Reddit(
#     client_id='w-kwRyPigyjYeG9DOiDc8g', 
#     client_secret='ZeDsvNH2YlpVH7F9wEWPkt5wkjLzqA',  
#     user_agent='sports_misinfo_script'  
# )

# subreddit = reddit.subreddit('sports+fantasyfootball') #2 subreddits 

# recent_posts = []
# for post in subreddit.new(limit= 5000):
#     created_date = datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S')
#     recent_posts.append({
#         'title': post.title,
#         'score': post.score,
#         'url': post.url,
#         'id': post.id,
#         'author': str(post.author),
#         'text': post.selftext,
#         'created_date': created_date,
#         'num_comments': post.num_comments,
#         'subreddit': post.subreddit.display_name,

#     })

# print(f"Fetched {len(recent_posts)} posts")

# for i, post in enumerate(recent_posts[:5]):
#     print(f"{i+1}. Title: {post['title']} | Score: {post['score']} | URL: {post['url']}")


# #-------------------------------
# #Save the data to a csv file
# fieldnames = ['title', 'score', 'url', 'id', 'author', 'text', 'created_date', 'num_comments', 'subreddit']

# with open('recent_sports_reddit_posts.csv', mode='w', newline='', encoding='utf-8') as csv_file:
#     writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

#     writer.writeheader()

#     for post in recent_posts:
#         writer.writerow(post)

# print("Data successfully saved to 'recent_sports_reddit_posts.csv'")
# print("Data saved")


In [3]:
import feedparser
import requests
from bs4 import BeautifulSoup

#add urls to this list to parse
url_list = [
        # #MBFC High Credibility-9 
        "https://www.espn.com/espn/rss/news", #ESPN top headlines
        "https://deadspin.com/rss/", #Deadspin 
        "https://feeds.abcnews.com/abcnews/sportsheadlines", #ABC news- espn sports
        "https://rss.nytimes.com/services/xml/rss/nyt/Sports.xml", #NYT
        "https://www.nytimes.com/athletic/rss/news/", #The Athletic- acquired by NYT
        "https://www.mlb.com/feeds/news/rss.xml", #MLB news 
        "https://www.reutersagency.com/feed/?best-topics=sports&post_type=best", # Reuters
        "https://sports.yahoo.com/rss/", #Yahoo News
        "https://www.cbssports.com/rss/headlines/", #CBS sports general headlines 

        #MBFC questionable sources or medium credibility- 5
        "https://notthebee.com/feed", #not the bee
        "https://uproxx.com/sports/feed/", #uproxx   
        "https://www.vibe.com/c/news/sports/feed/", #The vibe - medium cred 
        "https://moxie.foxnews.com/google-publisher/sports.xml", #fox news 
        "https://nypost.com/sports/feed/", #NY post - medium cred

        #NO MBFC RATING -11 
        "https://www.sportscollectorsdaily.com/feed/", #Sports Collectors Daily 
        "https://news.sportslogos.net/feed/", #SportsLogos.Net
        "https://www.sportingnews.com/us/rss", #sportingnews.com
        "https://www.sportskeeda.com/feed",#sportskeeda.com
        "https://sportsweez.com/feed/", #sportsweez
        "https://sportsbrackets.net/feed/", #sportsbracket
        "https://21sports.com/feed/", # 21 sports
        "https://www.essentiallysports.com/feed/", #essentiallysports
        "https://boxingnewsonline.net/feed/", #boxing news
        "https://www.thecoldwire.com/feed/", #the cold wire
        "https://sportsdark.com/feed/", #sports dark
         
]
    
entries = [] #list of dictionaries

#--------------DEFINE FUNCTION TO SCRAPE-------------------
def scrape_article_content(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")

        #extract content from common tags
        article_body = (
            soup.find("article") or
            soup.find("div", {"class": "post-content"}) or
            soup.find("div", class_="article-body") or
            soup.find("div", class_="article-content") or
            soup.find("section", class_="article-section") or
            soup.find("div", class_="main-content") or
            soup.find("div", class_="content-body")
        )
        
        if article_body:
            paragraphs = article_body.find_all("p")
        else:
            paragraphs = soup.find_all("p")

        #join all paragraphs into a single string
        article_content = " ".join(p.get_text() for p in paragraphs)
        return article_content.strip() if article_content else "No content found"

    except Exception as e:
        print(f"Failed to scrape content from {url}: {e}")
        return "Failed to fetch content"


#run the function to collect feeds and scrape
for url in url_list:
    feed = feedparser.parse(url)
    #feed_title= feed.feed.title
    feed_title = getattr(feed.feed, "title", None)

    
    for entry in feed.entries:
        entry_title = getattr(entry, "title", None)
        entry_link = getattr(entry, "link", None)
        entry_published_date = getattr(entry, "published", None)
        entry_summary = getattr(entry, "summary", None)
        entry_content = scrape_article_content(entry_link) #scrape
        
        entries.append({
            "feed_title": feed_title,
            "entry_title": entry_title,
            "entry_link": entry_link,
            "entry_published_date": entry_published_date,
            "entry_summary": entry_summary,
            "entry_content": entry_content,
        })

df = pd.DataFrame(entries)
#print(df)

df.to_csv('RSS_sports_feeds_11-20.csv', index=False)

Failed to scrape content from https://www.espn.com/nba/story/_/id/42468239/nba-power-rankings-okc-battles-west-magic-ascend-east: 403 Client Error: Forbidden for url: https://www.espn.com/nba/story/_/id/42468239/nba-power-rankings-okc-battles-west-magic-ascend-east
Failed to scrape content from https://www.espn.com/mlb/story/_/id/42512141/braves-chris-sale-wins-national-league-cy-young-award: 403 Client Error: Forbidden for url: https://www.espn.com/mlb/story/_/id/42512141/braves-chris-sale-wins-national-league-cy-young-award
Failed to scrape content from https://www.espn.com/wnba/story/_/id/42510300/caitlin-clark-play-unrivaled-league-source-says: 403 Client Error: Forbidden for url: https://www.espn.com/wnba/story/_/id/42510300/caitlin-clark-play-unrivaled-league-source-says
Failed to scrape content from https://www.espn.com/nfl/story/_/id/42508628/sources-jets-owner-proposed-benching-aaron-rodgers-sept-30: 403 Client Error: Forbidden for url: https://www.espn.com/nfl/story/_/id/4250

### Attempt to Scrape Links without RSS Feeds- not very effective 

In [9]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd

# uncredible_urls = [
#     "https://www.tellerreport.com/sports",
#     "https://www.newsbreak.com/mountain-view-ca-sports",
#     "https://newsrnd.com/sports",
#     "https://baltimorecitywire.com/stories/tag/53-sports"
# ]

# # Function to scrape article details
# def scrape_article(url):
#     try:
#         response = requests.get(url, timeout=10)
#         response.raise_for_status()
#         soup = BeautifulSoup(response.content, "html.parser")

#         # Extract the article title
#         title = soup.find("h1").get_text() if soup.find("h1") else soup.title.get_text()

#         # Extract the publication date (common in <time> or meta tags)
#         date = soup.find("time")
#         if date:
#             publication_date = date.get("datetime") or date.get_text()
#         else:
#             date_meta = soup.find("meta", {"name": "article:published_time"})
#             publication_date = date_meta["content"] if date_meta else "No date found"

#         # Extract the article content
#         content_container = (
#             soup.find("article") or
#             soup.find("div", class_=["post-content", "article-body", "article-content", "content-body"])
#         )
#         if content_container:
#             paragraphs = content_container.find_all("p")
#         else:
#             paragraphs = soup.find_all("p")

#         content = "\n".join(p.get_text() for p in paragraphs)

#         return {
#             "Title": title.strip(),
#             "Publication Date": publication_date.strip(),
#             "Content": content.strip()[:500] + "...",
#             "Link": url
#         }

#     except Exception as e:
#         print(f"Failed to scrape {url}: {e}")
#         return None

# articles = []
# for url in uncredible_urls:
#     print(f"Scraping: {url}")
#     article_details = scrape_article(url)
#     if article_details:
#         articles.append(article_details)

# for article in articles:
#     print("\n--- Article ---")
#     print(f"Title: {article['Title']}")
#     print(f"Date: {article['Publication Date']}")
#     print(f"Content Preview: {article['Content']}")
#     print(f"Link: {article['Link']}\n")

# df = pd.DataFrame(articles)
# output_file = "scraped_uncredible_articles.csv"
# df.to_csv(output_file, index=False)
# print(f"\nResults saved to {output_file}")

### Format DF to match DB table

In [123]:
from datetime import datetime
from dateutil import parser

df['publication_date'] = df['entry_published_date'].apply(lambda x: parser.parse(x).date()) #parse different date formats to date object format
dbdf = pd.read_csv('RSS_sports_feeds.csv')
dbdf['team_or_player'] = df['entry_title']
dbdf['source'] = df['feed_title']
dbdf['publication_date'] = df['publication_date'] 
dbdf['content'] = df['entry_summary']    #entry content if not empty? or keep as summary
dbdf['trust_score'] = 0.00               #CHANGE to score column
dbdf['classification'] = 'unknown'       #CHANGE to label column 
dbdf['link'] = df['entry_link']

#make a new df in the format of the DB table for easy inserting
sports_DB_df = dbdf[['team_or_player', 'source', 'publication_date', 'content', 'trust_score', 'classification', 'link']]
#save to a new CSV 
sports_DB_df.to_csv('formatted_sports_posts_for_DB.csv', index=False)
print("done")

done


C:\Users\tizia\Anaconda4\Lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


### Feature Computation Functions - use some for labeling and others for model

In [53]:
# !pip install textstat
from transformers import pipeline
import string
from textstat import flesch_reading_ease
import textstat
import re
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()


#functions for each feature we are considering
def word_count(text):
    return len(text.split())

def char_count(text):
    return len(text)

def avg_word_length(text):
    words = text.split()
    return sum(len(word) for word in words) / len(words) if words else 0

def all_capitalized_word_count(text):
    return sum(1 for word in text.split() if word.isupper())

def punctuation_count(text):
    return sum(1 for char in text if char in string.punctuation)

def exclamation_count(text):
    return text.count("!")

def question_mark_count(text):
    return text.count("?")

def readability_score(text): #high score = easier readability
    return flesch_reading_ease(text)

def smog_index(text): #readability based on the number of complex words (3+ syllables)
    return textstat.smog_index(text)

def count_quotation_pairs(text): 
    double_quotes = re.findall(r'"[^"]+"', text) #Finds all sections of text in double quotes
    return len(double_quotes)

def variety_of_vocabularity(text):
    words = text.split()
    unique_words = set(words)
    return len(unique_words) / len(words) if words else 0


sensational_words = ["shocking","shock", "amazing", "unbelievable", "you won’t believe", "incredible", "stunning",\
                    "astounding", "breathtaking", "outstanding", "thrilling", "must see", "terrible", "awful",\
                     "fantastic", "horrible", "remarkable"]
def count_sensational_words(text):
    text = text.lower()
    return sum(1 for word in sensational_words if word in text)


absolute_words = ["always", "never", "clearly", "obviously", "definitely", "everyone", "nobody", "all",\
              "none", "blatantly", "undoubtedly", "must", "should"]
def count_absolute_words(text):
    words = text.lower().split()
    return sum(1 for word in words if word in absolute_words)


factuality_words = ["proven", "irrefutable", "unarguable", "unquestionably", "certainly", "definitely",\
                    "undeniable"]
def count_factuality_words(text):
    words = text.lower().split()
    return sum(1 for word in words if word in factuality_words)


subjective_words = ["believe", "think", "feel", "prefer", "seems", "wish"]
def subjectivity_score(text):
    words = text.lower().split()
    subjective_count = sum(1 for word in words if word in subjective_words)
    return subjective_count / len(words) if len(words) > 0 else 0

objective_words = ["reported", "measured", "confirmed", "analyzed", "observed", "recorded",\
                   "found", "documented", "verified", "tested", "studied", "calculated", "noted",\
                   "established", "evidence", "fact", "data", "statistics", "demonstrated", "shown",\
                   "results", "result", "evidence-based", "peer-reviewed", "sampled", "quantified",\
                   "evaluated", "experimented", "investigated"]
def objective_score(text):
    words = text.lower().split()
    objective_count = sum(1 for word in words if word in objective_words)
    return objective_count / len(words) if len(words) > 0 else 0


def count_numerics(text):
    numerical_count = len(re.findall(r'\d+', text)) 
    return numerical_count


pipe_finnews = pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
def sentiment(text):
    result = pipe_finnews(text)
    return result[0]["label"] 


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\tizia\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### Define Labeling Heuristic Function

In [55]:
def calculate_credibility_score(headline, source_credibility):
    # Source credibility weights
    SOURCE_CREDIBILITY = {
        'high': 1.0,
        'unknown': 0,
        'medium': -0.5,
        'low': -1.0
    }
    
    # Top heuristics and their weights (based on the difference column)
    heuristics = {
        'smog_index': {
            'weight': 9.448,  # From the Difference column
            'true_avg': 0.124, # From the True_Avg column
            'false_avg': 1.169 # From the False_Avg column
        },
        'factuality_words_count': {
            'weight': 8.200,  
            'true_avg': 0.000263,
            'false_avg': 0.002158
        },
        'exclamation_count': {
            'weight': 6.326, 
            'true_avg': 0.0192,
            'false_avg': 0.1215
        },
        'subjectivity_score': {
            'weight': 4.308,  
            'true_avg': 0.000373,
            'false_avg': 0.001606
        },
        'question_mark_count': {
            'weight': 4.285,  
            'true_avg': 0.0327,
            'false_avg': 0.1403
        },
        'count_numerics': {
            'weight': 3.089,  
            'true_avg': 0.7496,
            'false_avg': 0.2427
        }
    }   
    
    # Calculate metrics for each of the heuristics we're using
    metrics = {
        'smog_index': smog_index(headline),
        'factuality_words_count': count_factuality_words(headline),
        'exclamation_count': exclamation_count(headline),
        'subjectivity_score': subjectivity_score(headline),
        'question_mark_count': question_mark_count(headline),
        'count_numerics': count_numerics(headline)
    }

    # Initialize the credibility score
    credibility_score = 0
    
    # Calculate scores for each heuristic
    for metric, values in heuristics.items():
        true_avg = values['true_avg']
        false_avg = values['false_avg']
        weight = values['weight']
        current_value = metrics[metric]
        
        # Calculate which average the current value is closer to
        dist_to_true = abs(current_value - true_avg)
        dist_to_false = abs(current_value - false_avg)
        
        # Calculate percentage difference from the midpoint
        midpoint = (true_avg + false_avg) / 2            
        percent_diff = abs(current_value - midpoint) / midpoint
        
        # Add or subtract from score based on which average it's closer to
        if dist_to_true < dist_to_false:
            credibility_score += weight * percent_diff
        else:
            credibility_score -= weight * percent_diff
        
        # print(f"{metric}: {current_value:.4f}\n\tWeight: {weight:.2f}\n\tPercent Difference: {percent_diff:.4f}\n\tCredibility Score: {credibility_score:.2f}")
    
    # Add source credibility modifier
    if source_credibility: 
        source_credibility_score = SOURCE_CREDIBILITY.get(source_credibility.lower(), SOURCE_CREDIBILITY['unknown'])
        weight = 10
        credibility_score += (source_credibility_score * weight)
        # print(f"source_credibility: {source_credibility_score:.2f}\n\tWeight: {weight:.2f}\n\tCredibility Score: {credibility_score:.2f}")

    return credibility_score


### Define Source credibilty of each source and run Labeling Heuristic Function

In [72]:
#------------------------Determine source credibility-------------------------------------------

all_articles_df = pd.read_csv('All_RSS_articles.csv')
print(all_articles_df["feed_title"].unique())
    
high_cred= [
    'www.espn.com - TOP',
    'Deadspin > Sports News Without Fear, Favor or Compromise',
    'Yahoo! Sports - News, Scores, Standings, Rumors, Fantasy Games',
    'MLB News',
    'ABC News: Sports',
    'Latest Headlines- The Athletic',
    'NYT > Sports',
    'CBSSports.com Headlines',
]

medium_cred=[
    'Sports', #vibe
    'NY Post Sports – Latest News, Scores, Stats & Videos' 'ABC News: Sports',
]  

low_cred=[
    'Sports – UPROXX',
    'Not the Bee',
    'Latest Sports News Today on Fox News', #foxnews.com
]

no_cred=[
    'SportsLogos.Net News',
    'Sporting News RSS', 
    'Sports Weez', 
    'Sports Brackets',
    '21Sports.com', 
    'Boxing News', 
    'The Cold Wire', 
    'Sportsdark',
    'EssentiallySports', 
    'Sportskeeda', 
    'Sports Collectors Daily',
    'FOX Sports Digital', #foxsports.com
]

def determine_source_credibility(feed_title):
    if feed_title in high_cred:
        return "high"
    elif feed_title in medium_cred:
        return "medium"
    elif feed_title in low_cred:
        return "low"
    else:
        return "unknown"  


#Create a new column for source_credibility based on the feed_title
all_articles_df["source_credibility"] = all_articles_df["feed_title"].apply(determine_source_credibility)


CREDIBLE_VALUE= 30 ##PLACEHOLDER THRESHOLD NEEDED TO BE CREDIBLE ******************CHANGE*********
credibility_scores = []
credibility_labels = []
for index, row in all_articles_df.iterrows():
    #get the headline and source_credibility
    headline = row["entry_title"]
    source_credibility = row["source_credibility"]
    
    #Calculate the credibility score 
    score= calculate_credibility_score(headline, source_credibility)
    credibility_scores.append(score) #add to list

    if score>= CREDIBLE_VALUE:
        credibility_labels.append("credible")
    else:
        credibility_labels.append("uncredible")


all_articles_df["credibility_score"] = credibility_scores
all_articles_df["credibility_label"] = credibility_labels

all_articles_df.to_csv("labeled_RSS_posts.csv")
print("saved")

['NY Post Sports – Latest News, Scores, Stats & Videos' 'ABC News: Sports'
 'Latest Headlines - The Athletic' 'www.espn.com - TOP'
 'Deadspin > Sports News Without Fear, Favor or Compromise'
 'Yahoo! Sports - News, Scores, Standings, Rumors, Fantasy Games'
 'Sports – UPROXX' 'Sports' 'Latest Sports News Today on Fox News'
 'SportsLogos.Net News' 'Sporting News RSS' 'Sports Weez'
 'Sports Brackets' '21Sports.com' 'Boxing News' 'The Cold Wire'
 'Sportsdark' 'EssentiallySports' 'Sportskeeda' 'MLB News' 'Not the Bee'
 'FOX Sports Digital' 'Sports Collectors Daily']
saved


In [74]:
all_articles_df #labeled dataframe 

,feed_title,entry_title,entry_link,entry_published_date,entry_summary,entry_content,publication_date,source_credibility,credibility_score,credibility_label
0,"NY Post Sports – Latest News, Scores, Stats & ...",Ohio State center suffers season-ending Achill...,https://nypost.com/2024/11/19/sports/ohio-stat...,"Tue, 19 Nov 2024 23:21:07 -0500",McLaughlin had been Ohio State’s starting cent...,Ohio State center Seth McLaughlin suffered a t...,2024-11-19,unknown,29.478000,uncredible
1,"NY Post Sports – Latest News, Scores, Stats & ...",Guardians’ Stephen Vogt and Brewers’ Pat Murph...,https://nypost.com/2024/11/19/sports/indians-s...,"Tue, 19 Nov 2024 22:46:09 -0500",Pat Murphy won NL Manager of the Year on Tuesd...,Milwaukee’s Pat Murphy and Cleveland’s Stephen...,2024-11-19,unknown,29.478000,uncredible
2,"NY Post Sports – Latest News, Scores, Stats & ...",Cam Johnson goes off for 34 points as shorthan...,https://nypost.com/2024/11/19/sports/cam-johns...,"Tue, 19 Nov 2024 22:43:18 -0500",The Nets found a way to replicate Thomas’ prod...,The status of leading scorer Cam Thomas gradua...,2024-11-19,unknown,35.703940,credible
3,"NY Post Sports – Latest News, Scores, Stats & ...",The possible candidates to replace Joe Douglas...,https://nypost.com/2024/11/19/sports/the-early...,"Tue, 19 Nov 2024 22:19:51 -0500",The Jets fired Joe Douglas on Tuesday and now ...,The Jets fired Joe Douglas on Tuesday and now ...,2024-11-19,unknown,29.478000,uncredible
4,ABC News: Sports,Mark Scheifele scores a hat trick and league-l...,https://abcnews.go.com/Sports/wireStory/mark-s...,"Tue, 19 Nov 2024 23:54:18 -0500","Mark Scheifele had his ninth career hat trick,...","Mark Scheifele had his ninth career hat trick,...",2024-11-19,high,51.929879,credible
...,...,...,...,...,...,...,...,...,...,...
2365,SportsLogos.Net News,"NFL Week 9 Uniform Schedule: Giants, Eagles, P...",https://news.sportslogos.net/2024/11/03/nfl-we...,"Sun, 03 Nov 2024 16:25:35 +0000",Our weekly series that looks at every uniform ...,The midway point of the 2024-25 season is here...,NaN,unknown,35.703940,credible
2366,SportsLogos.Net News,QMJHL’s Quebec Remparts Pay Homage to Beloved ...,https://news.sportslogos.net/2024/11/02/qmjhls...,"Sat, 02 Nov 2024 15:00:00 +0000",Jerseys worn Friday vs. Sherbrooke Phoenix ins...,The Quebec Remparts paid homage to one of thei...,NaN,unknown,29.478000,uncredible
2367,SportsLogos.Net News,Oregon Ducks Unveil Final “Generation O” Unifo...,https://news.sportslogos.net/2024/11/02/oregon...,"Sat, 02 Nov 2024 05:03:33 +0000",Oregon unveiled the fifth and final combinatio...,The Oregon football program unveiled the fifth...,NaN,unknown,29.478000,uncredible
2368,SportsLogos.Net News,Illinois Fighting Illini Unveil Military Appre...,https://news.sportslogos.net/2024/11/02/illino...,"Sat, 02 Nov 2024 04:02:08 +0000",Illinois will wear a special helmet design dur...,The Illinois football program will wear a spec...,NaN,unknown,29.478000,uncredible


### Data Exploration
-get number of credible labels vs uncredible counts

-number of each source (using feed title)


## Train a model using glove and additional features, source split to eval if model can generalize

### Turn each headline into a glove embedding

In [ ]:
#article vec is mean of token's embeddings







### Prep the data into train and test by source split

In [98]:
from sklearn.model_selection import train_test_split
import numpy as np

def extract_features(headline): #features not in heuristic for labels 
    return np.array([
        word_count(headline),
        char_count(headline),
        avg_word_length(headline),
        all_capitalized_word_count(headline),
        punctuation_count(headline),
        readability_score(headline),
        count_quotation_pairs(headline),
        variety_of_vocabularity(headline),
        count_sensational_words(headline),
        count_absolute_words(headline),
        objective_score(headline),
        sentiment(headline)
    ])

data = pd.read_csv('labeled_RSS_posts.csv') #get labeled data

#Split up sources randomly, aka by feed_title
sources = data["feed_title"].unique()
train_sources, test_sources = train_test_split(sources, test_size=0.2, random_state=42)

#Create train and test splits based on sources
train_data = data[data["feed_title"].isin(train_sources)]
test_data = data[data["feed_title"].isin(test_sources)]   ## ADJUST TEST SIZE BASED ON ENTRIES IN EACH OF THESE??
#---------------------------------------

# #Apply embeddings and feature calculations to each headline, combine features and glove embeddings
# X_train = np.array([
#     np.hstack((GLOVE_FUNCTION(row["entry_title"], glove_model), extract_features(row["entry_title"])))
#     for index, row in train_data.iterrows()
# ])

# # Prepare testing data
# X_test = np.array([
#     np.hstack((GLOVE_FUNCTION(row["entry_title"], glove_model), extract_features(row["entry_title"])))
#     for index, row in test_data.iterrows()
# ])

y_train = (train_data["credibility_label"] == "credible").astype(int).values #1=credible, 0= uncredible
y_test = (test_data["credibility_label"] == "credible").astype(int).values



### Random Forest model

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# model = RandomForestClassifier(random_state=42)




